In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import hydra
from hydra.utils import get_original_cwd
from omegaconf import DictConfig
from config import run_training
from data import build_loaders
from plot import setup_dirs, dump_results
import os
import pickle


def build_dataset_path(seed, n_samples, dim, eps, curv, transform_dim):
    template = 'dim={},n_samples={},eps={},transform_dim={},curv={},seed={}.pkl'
    return template.format(int(dim), int(n_samples), eps, transform_dim, curv, seed)


def train(cfg: DictConfig) -> None:
    cwd = os.getcwd()
    print("Working directory : {}".format(cwd))

    dataset_path = build_dataset_path(**cfg['dataset_params'])
    dataset_path = os.path.join(cwd, 'datasets', dataset_path)

    with open(dataset_path, 'rb') as f:
        datasets = pickle.load(f)
    loaders = build_loaders(datasets, cfg['bs'], cfg['num_workers'])

    results, run = run_training(loaders=loaders,
                                cfg=cfg,
                                **cfg)
    folder_names = set(datasets.keys()) | set(results.keys())
    base_dir = os.getcwd()
    setup_dirs(folder_names, base_dir)
    dump_results(datasets, results, base_dir, run)
    if run is not None:
        run.stop()

# build datasets

In [7]:
from hydra import initialize, compose
from data import build_datasets

base_overrides = ['datasets_folder=datasets']
# multirun = [['dim=i']]

# for additional_ovrs in multirun:
#     bo_copy = set(base_overrides)
#     bo_copy.extend(additional_ovrs)

with initialize(config_path="conf"):
    cfg = compose(config_name="dataset_generation.yaml", overrides=base_overrides)
    print(cfg)
    build_datasets(**cfg)

{'seed': 777, 'n_samples': 100000.0, 'dim': 2, 'eps': 0.01, 'curv': 1, 'transform_dim': '${.dim}', 'datasets_folder': 'datasets'}


# run model training

In [ ]:
from hydra import initialize, compose

overrides = [
    'epochs=200',
    'model.head.hidden_dims=[320,320,320]'
]

with initialize(config_path="conf"):
    cfg = compose(config_name="config.yaml", overrides=overrides)
    train(cfg)

Working directory : /scidatalg/jkoc/hyper-dl
https://app.neptune.ai/jkociniak/hyperbolic-dist-pred/e/HYPERDIST-2704
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Found device: cuda
[1] train loss (mean of batch losses): 1.8063308995383127
[1] val loss: 0.1921956802368164
[2] train loss (mean of batch losses): 0.1436576313018799
[2] val loss: 0.13269782819747925
[3] train loss (mean of batch losses): 0.1218020978655134
[3] val loss: 0.12143896350860596
[4] train loss (mean of batch losses): 0.11299987373352051
[4] val loss: 0.11397585487365723
[5] train loss (mean of batch losses): 0.10662208243778774
[5] val loss: 0.10705919742584229
[6] train loss (mean of batch losses): 0.09974978813443865
[6] val loss: 0.1006148281097412
[7] train loss (mean of batch losses): 0.09423584240504673
[7] val loss: 0.09569735169410705
[

In [4]:
!cat conf/layers/2input_hFFN.yaml

name: DoubleInputHyperbolicFFN
input_dim: 2
hidden_dims: []
output_dim: 2
bias: False
curv: 1
activation: None
skips: False